In [ ]:
import ee
import geemap 
import matplotlib.pyplot as plt
from datetime import timedelta
import requests
from datetime import datetime
from collections import defaultdict
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import seaborn as sns
import xarray as xr
import eccodes
import cdsapi
import cfgrib
import glob
from collections import defaultdict

# Functions

## Extract data

In [ ]:
# Initialize Earth Engine
ee.Initialize()

# Function to extract temperature values from ERA5 dataset
def extract_values(latitude, longitude, start_date, end_date, dataset_name, property_name):
    dataset = ee.ImageCollection(dataset_name).select(property_name)
    station_coordinates = ee.Geometry.Point([longitude, latitude])

    # Split date range into smaller intervals
    date_ranges = pd.date_range(start=start_date, end=end_date, freq='1YE')
    
    df_list = []
    for i in range(len(date_ranges)-1):
        filtered_dataset = dataset.filterDate(date_ranges[i].strftime('%Y-%m-%d'), date_ranges[i+1].strftime('%Y-%m-%d'))
        
        def extract_values(img):
            mean_value = img.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=station_coordinates,
                scale=1,
                maxPixels=1e9
            ).get(property_name)
    
            return ee.Feature(None, {
                'date': ee.Date(img.get('system:time_start')).format('YYYY-MM-dd'),
                'value': mean_value
            })
    
        values = filtered_dataset.map(extract_values)
        value_list = values.getInfo()['features']
        
        df = pd.DataFrame([feature['properties'] for feature in value_list])
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df.dropna(subset=['value'], inplace=True)
        
        df_list.append(df)
    
    return pd.concat(df_list)

# Function to extract temperature values from climate station data
def extract_climate_station_values(climateID, start_date, end_date, layer_name, property_name):
    url = f"https://api.weather.gc.ca/collections/{layer_name}/items?datetime={start_date}/{end_date}&CLIMATE_IDENTIFIER={climateID}&f=json&sortby=LOCAL_DATE&limit=20000"
    print(url)
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Failed to retrieve data: {response.status_code}")

    dates = []
    y_axis = []

    for feature in data['features']:
        dates.append(pd.to_datetime(feature['properties']['LOCAL_DATE']))
        y_axis.append(feature['properties'][property_name])

    dates_to_remove = []
    for date in pd.date_range(start=start_date, end=end_date, freq='D'):
        if date not in dates:
            dates.append(date)
            y_axis.append(None)
            dates_to_remove.append(date)    

    dates, y_axis = zip(*sorted(zip(dates, y_axis)))
    
    return dates, y_axis

def extraer_datos_nieve(latitud_objetivo, longitud_objetivo):
  todos_los_datos = [] 
  ruta_archivos = "D:\\Xavi\\Proyecto Canada\\snow_NSIDC\\snow_*.txt"
  chunksize = 100000

  for archivo in glob.glob(ruta_archivos):
    for chunk in pd.read_csv(archivo, sep="\t", header=None, chunksize=chunksize, skiprows=1):
      # Asignar nombres de columna
      chunk.columns = ["fecha", "longitud", "latitud", "profundidad_nieve"]

      chunk["fecha"] = chunk["fecha"].astype(str)  # Convertir a string
      chunk["fecha"] = chunk["fecha"].str[:-2]   # Eliminar los últimos dos ceros
      chunk["fecha"] = pd.to_datetime(chunk["fecha"], format="%Y%m%d") 

      # Calcular la distancia a las coordenadas objetivo
      chunk["distancia"] = ((chunk["latitud"] - latitud_objetivo)**2 + 
                            (chunk["longitud"] - longitud_objetivo)**2)**0.5

      # Encontrar la fila con la menor distancia en el chunk
      fila_cercana = chunk.loc[chunk["distancia"].idxmin()]

      # Agregar la fecha y profundidad de nieve a la lista
      todos_los_datos.append([fila_cercana["fecha"], fila_cercana["profundidad_nieve"]])

  # Crear un DataFrame con los datos recopilados
  df_final = pd.DataFrame(todos_los_datos, columns=["fecha", "profundidad_nieve"])
  return df_final

## Calculate metrics

In [ ]:
# Calculate R^2, MAE, and RMSE
def calculate_metrics(y_true, y_pred):
    y_true= np.array(y_true)
    y_pred = np.array(y_pred) 
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    bias = np.mean(np.array(y_true) - np.array(y_pred))
    return r2, mae, rmse, bias

def separate_data_by_year(dates, values):
  year_data = {}
  for date, value in zip(dates, values):
    year = str(date.year)
    if year not in year_data:
      year_data[year] = []
    year_data[year].append(value)  # Store only the value
  return year_data

def separate_data_by_month(dates, values):
  month_data = {}
  for date, value in zip(dates, values):
    month = date.month
    if month not in month_data:
      month_data[month] = []
    month_data[month].append(value) 
  return month_data

def process_data_by_year(dates, values, additional_values=None):
  year_data = separate_data_by_year(dates, values)
  if additional_values:
    for year, extra_vals in additional_values.items():
      if year in year_data:
        year_data[year].extend(extra_vals)  # Append if year exists
      else:
        year_data[year] = extra_vals.copy()  # Add year with values if it doesn't exist
  return year_data

def process_data_by_month(dates, values, additional_values=None):
  month_data = separate_data_by_month(dates, values)
  if additional_values:
    for month, extra_vals in additional_values.items():
      if month in month_data:
        month_data[month].extend(extra_vals)  # Append additional values
  return month_data

def calculate_metrics_by_year(year_data, year_predictions):
  metrics_by_year = {}
  for year in year_data:
    if year in year_predictions:
      y_true_year = year_data[year]
      y_pred_year = year_predictions[year]
      metrics = calculate_metrics(y_true_year, y_pred_year)
      metrics_by_year[year] = {
          "r2": metrics[0],
          "mae": metrics[1],
          "rmse": metrics[2],
          "bias": metrics[3]
      }
  return metrics_by_year

def calculate_metrics_by_month(month_data, month_predictions):
  metrics_by_month = {}
  for month in month_data:
    if month in month_predictions:
      y_true_month = month_data[month]
      y_pred_month = month_predictions[month]
      metrics = calculate_metrics(y_true_month, y_pred_month)
      metrics_by_month[month] = {
          "r2": metrics[0],
          "mae": metrics[1],
          "rmse": metrics[2],
          "bias": metrics[3]
      }
  return metrics_by_month

# Parameters

## Skin Temperature

### Metrics per location

In [ ]:
#Yukon Territory
Parameter= 'Daily Skin Temperature'

locations = [
    {'name': "YGS_WatsonLake_BH1",'latitude': 60.151, 'longitude': -128.885, 'climateID': None}, 
    {'name': "YGS_MarshLake_BH1",'latitude': 60.561, 'longitude': -134.445, 'climateID': None},
    {'name': "YGS_Cowley_Creek_BH1",'latitude': 60.593, 'longitude': -134.905, 'climateID': None},
    {'name': "YGS_Cowley_Creek_BH2",'latitude': 60.593, 'longitude': -134.904, 'climateID': None},
    {'name': "YGS_FishLk_BH1",'latitude': 60.653, 'longitude': -135.260, 'climateID': None},
    {'name': "YGS_Hamilton_Blvd_HB_BH1",'latitude': 60.695, 'longitude': -135.096, 'climateID': None},
    {'name': "YGS_HJ_BH2",'latitude': 60.772, 'longitude': -137.495, 'climateID': None},
    {'name': "YGS_HJ_BH5",'latitude': 60.783, 'longitude': -137.531, 'climateID': None},
    {'name': "YGS_HJ_BH6",'latitude': 60.786, 'longitude': -137.537, 'climateID': None},
    {'name': "YGS_HiddenValley",'latitude': 60.830, 'longitude': -135.200, 'climateID': None},
    {'name': "YGS_TakhiniValley",'latitude': 60.843, 'longitude': -135.673, 'climateID': None},
    {'name': "YGS_Takhini_Thawslump",'latitude': 60.856, 'longitude': -135.515, 'climateID': None},
    {'name': "YGS_Nordenskiold_BH2",'latitude': 61.856, 'longitude': -136.109, 'climateID': None},
    {'name': "YGS_Nordenskiold_BH3",'latitude': 61.857, 'longitude': -136.109, 'climateID': None},
    {'name': "YGS_CanolRd",'latitude': 61.924, 'longitude': -132.571, 'climateID': None},
    {'name': "YGS_Carmacks_BH3",'latitude': 62.091, 'longitude': -136.299, 'climateID': None},
    {'name': "YGS_Faro",'latitude': 62.223, 'longitude': -133.342, 'climateID': None},
    {'name': "HPW_Shakwak_1_toe",'latitude': 62.336 , 'longitude': -140.834, 'climateID': None},
    {'name': "YGS_BeaverCreek",'latitude': 62.337, 'longitude': -140.835, 'climateID': None},
    {'name': "HPW_Shakwak_orig_berm",'latitude': 62.338, 'longitude': -104.835, 'climateID': None},
    {'name': "HPW_Shakwak_orig_centre",'latitude': 62.338, 'longitude': -140.834, 'climateID': None},
    {'name': "YGS_BeaverCreek_BH1",'latitude': 62.371, 'longitude': -140.876, 'climateID': None},
    {'name': "YGS_BeaverCreek_BH3",'latitude': 62.384, 'longitude': -140.870, 'climateID': None},
    {'name': "YGS_BeaverCreek_BH2",'latitude': 62.400, 'longitude': -140.866, 'climateID': None},
    {'name': "Coffee_KPBH4",'latitude': 62.858, 'longitude': -139.406, 'climateID': None},
    {'name': "Coffee_KPBH1",'latitude': 62.863, 'longitude': -139.399, 'climateID': None},
    {'name': "HPW_StewartCrossing",'latitude': 63.374, 'longitude': -136.677, 'climateID': None}, 
    {'name': "YGS_DawsonDump",'latitude': 64.031, 'longitude': -139.294, 'climateID': None},
    {'name': "YGS_DawsonSchool",'latitude': 64.061, 'longitude': -139.430, 'climateID': None},
    {'name': "AAM_ClintonCk_BH18-08",'latitude': 64.452, 'longitude': -140.728, 'climateID': None},
    {'name': "AAM_ClintonCk_BH18-15",'latitude': 64.456, 'longitude': -140.715, 'climateID': None},
    {'name': "YGS_USArray_I30M_MtDempster",'latitude': 65.222, 'longitude': -136.376, 'climateID': None},
    {'name': "YGS_Dempster_KM190",'latitude': 65.326, 'longitude': -138.240, 'climateID': None},
    {'name': "YGS_Ogilvie_BH1",'latitude': 65.664, 'longitude': -138.134, 'climateID': None},
    {'name': "YGS_USArray_H31M_PeelRiver",'latitude': 65.805, 'longitude': -134.342, 'climateID': None},
    {'name': "YGS_USArray_H29_Whitestone",'latitude': 66.219, 'longitude': -138.368, 'climateID': None},    
    {'name': "YGS_EagleRiver",'latitude': 66.444, 'longitude': -136.708, 'climateID': None},
    {'name': "YGS_Dempster_KM934-2",'latitude': 66.519, 'longitude': -136.512, 'climateID': None},
    {'name': "YGS_Dempster_KM934-1",'latitude': 66.520, 'longitude': -136.512, 'climateID': None},
    {'name': "YGS_USArray_G29M_PineCk",'latitude': 66.911, 'longitude': -138.022, 'climateID': None},
    {'name': "YGS_USArray_E29M_BlowRiver",'latitude': 68.388, 'longitude': -137.896, 'climateID': None}
]

L=[]
R2_ERA5=[]
MAE_ERA5=[]
RMSE_ERA5=[]
BIAS_ERA5=[]
R2_ERA5_LAND=[]
MAE_ERA5_LAND=[]
RMSE_ERA5_LAND=[]
BIAS_ERA5_LAND=[]
R2_THERMAL=[]
MAE_THERMAL=[]
RMSE_THERMAL=[]
BIAS_THERMAL=[]
R2_MODIS=[]
MAE_MODIS=[]
RMSE_MODIS=[]
BIAS_MODIS=[]

# Loop over each location and plot temperature data
for location in locations:
    #ERA5-LAND - 11.1 km
    df_era5_land = extract_values(location['latitude'], location['longitude'], '1950-01-02', '2024-01-31','ECMWF/ERA5_LAND/DAILY_AGGR', 'skin_temperature')
    df_era5_land_dates=pd.to_datetime(df_era5_land['date'])
    df_era5_land_value=df_era5_land['value']-273.15
    df_era5_land_dates, df_era5_land_value = zip(*sorted(zip(df_era5_land_dates, df_era5_land_value)))
    #ERA5 - 27.8 km
    years = list(range(2000, 2025))
    skin_temperature_by_year = []
    for year in years:
        ds_year = xr.open_dataset(f'era5_skt{year}.grib', engine='cfgrib')
        ds_year_by_day = ds_year['skt'].resample(time='D').mean()
        skin_temperature_by_year.append(ds_year_by_day)
        ds_year.close()
    skin_temperature_combined = xr.concat(skin_temperature_by_year, dim='time')
    skin_temperature = skin_temperature_combined.sel(latitude=location['latitude'], longitude=location['longitude'], method='nearest') - 273.15
    df_era5_dates= pd.to_datetime(skin_temperature.time)
    df_era5_value=skin_temperature.values
    df_era5_dates, df_era5_value = zip(*sorted(zip(df_era5_dates, df_era5_value)))
    #INFO THERMAL - 1 km
    df_thermal = extract_values(location['latitude'], location['longitude'], '2012-01-19', '2024-03-01','NOAA/VIIRS/001/VNP21A1D','LST_1KM')
    df_thermal_dates=pd.to_datetime(df_thermal['date'])
    df_thermal_value=df_thermal['value']-273.15
    df_thermal_dates, df_thermal_value = zip(*sorted(zip(df_thermal_dates, df_thermal_value)))
    #INFO MODIS - 1 km
    df_modis=extract_values(location['latitude'], location['longitude'], '2000-02-24', '2024-03-24','MODIS/061/MOD11A1','LST_Day_1km')
    df_modis_dates=pd.to_datetime(df_modis['date'])
    df_modis_value=(df_modis['value']*0.02)-273.15
    df_modis_dates, df_modis_value = zip(*sorted(zip(df_modis_dates, df_modis_value)))
    #CLIMATE STATIONS
    # Ruta al archivo CSV
    file_path = f"{location['name']}.csv"
    data = pd.read_csv(file_path, sep=",")
    data["Date"] = pd.to_datetime(data["Date"])
    station_dates=data["Date"]
    station_value=data["0__m"]
    station_dates,station_value = zip(*sorted(zip(station_dates,station_value)))
    mask = np.isnan(station_value)
    non_nan_indices = np.logical_not(mask)
    # Utiliza los índices no NaN para obtener los valores y fechas correspondientes
    station_value = np.array(station_value)[non_nan_indices]
    station_dates = np.array(station_dates)[non_nan_indices]


    if len(station_dates) == 0:
        continue
    else:
        #Remove dates not present in both datasets. Siempre la estacion va a tener menos data.
        common_dates_era5 = set(df_era5_dates).intersection(set(station_dates)) #se extraen las fechas que tienen iguales
        common_dates_era5_land=set(df_era5_land_dates).intersection(set(station_dates))
        common_dates_thermal=set(df_thermal_dates).intersection(set(station_dates))
        common_dates_modis=set(df_modis_dates).intersection(set(station_dates))
        
        #VALORES ERA5
        era5_dates=[]
        era5_value=[]
        for a,b in zip(df_era5_dates,df_era5_value):
            if a in common_dates_era5:
                era5_dates.append(a)
                era5_value.append(b)
        st_dates_era5=[]
        st_value_era5=[]
        for a,b in zip(station_dates,station_value):
            if a in common_dates_era5:
                st_dates_era5.append(a)
                st_value_era5.append(b)
    
        #VALORES ERA5-LAND
        era5_land_dates=[]
        era5_land_value=[]
        for a,b in zip(df_era5_land_dates,df_era5_land_value):
            if a in common_dates_era5_land:
                era5_land_dates.append(a)
                era5_land_value.append(b)
        st_dates_era5_land=[]
        st_value_era5_land=[]
        for a,b in zip(station_dates,station_value):
            if a in common_dates_era5_land:
                st_dates_era5_land.append(a)
                st_value_era5_land.append(b)
            
        #VALORES THERMAL IMAGES
        thermal_dates=[]
        thermal_value=[]
        for a,b in zip(df_thermal_dates,df_thermal_value):
            if a in common_dates_thermal:
                thermal_dates.append(a)
                thermal_value.append(b)
        st_dates_thermal=[]
        st_value_thermal=[]
        for a,b in zip(station_dates,station_value):
            if a in common_dates_thermal:
                st_dates_thermal.append(a)
                st_value_thermal.append(b)

        #VALORES MODIS
        modis_dates=[]
        modis_value=[]
        for a,b in zip(df_modis_dates,df_modis_value):
            if a in common_dates_modis:
                modis_dates.append(a)
                modis_value.append(b)
        st_dates_modis=[]
        st_value_modis=[]
        for a,b in zip(station_dates,station_value):
            if a in common_dates_modis:
                st_dates_modis.append(a)
                st_value_modis.append(b)
                
        # Calculate metrics
        r2_era5, mae_era5, rmse_era5, bias_era5 = calculate_metrics(st_value_era5, era5_value)
        r2_era5_land, mae_era5_land, rmse_era5_land, bias_era5_land = calculate_metrics(st_value_era5_land, era5_land_value)
        r2_thermal, mae_thermal, rmse_thermal, bias_thermal=calculate_metrics(st_value_thermal, thermal_value)
        r2_modis, mae_modis, rmse_modis, bias_modis=calculate_metrics(st_value_modis, modis_value)
        #if r2_era5<0:
         #   r2_era5=np.nan
        #if r2_era5_land<0:
        #    r2_era5_land=np.nan
        print(f"Metrics ERA5 for {location['name']} - R^2: {r2_era5:.2f}, MAE: {mae_era5:.2f}, RMSE: {rmse_era5:.2f}, Bias: {bias_era5:.2f}")
        print(f"Metrics ERA5-LAND for {location['name']} - R^2: {r2_era5_land:.2f}, MAE: {mae_era5_land:.2f}, RMSE: {rmse_era5_land:.2f}, Bias: {bias_era5_land:.2f}")
        print(f"Metrics VNP21A1D for {location['name']} - R^2: {r2_thermal:.2f}, MAE: {mae_thermal:.2f}, RMSE: {rmse_thermal:.2f}, Bias: {bias_thermal:.2f}")
        print(f"Metrics MODIS for {location['name']} - R^2: {r2_modis:.2f}, MAE: {mae_modis:.2f}, RMSE: {rmse_modis:.2f}, Bias: {bias_modis:.2f}")
        L.append(location['name'])
        R2_ERA5.append(r2_era5)
        MAE_ERA5.append(mae_era5)
        RMSE_ERA5.append(rmse_era5)
        BIAS_ERA5.append(bias_era5)
        R2_ERA5_LAND.append(r2_era5_land)
        MAE_ERA5_LAND.append(mae_era5_land)
        RMSE_ERA5_LAND.append(rmse_era5_land)
        BIAS_ERA5_LAND.append(bias_era5_land)
        R2_THERMAL.append(r2_thermal)
        MAE_THERMAL.append(mae_thermal)
        RMSE_THERMAL.append(rmse_thermal)
        BIAS_THERMAL.append(bias_thermal)
        R2_MODIS.append(r2_modis)
        MAE_MODIS.append(mae_modis)
        RMSE_MODIS.append(rmse_modis)
        BIAS_MODIS.append(bias_modis)

# Set the positions of the points on the x-axis
x = np.arange(len(L))

# Define the metrics and corresponding data lists
metrics = ['R2', 'MAE', 'RMSE', 'BIAS']
data_ERA5 = [R2_ERA5, MAE_ERA5, RMSE_ERA5, BIAS_ERA5]
data_ERA5_LAND = [R2_ERA5_LAND, MAE_ERA5_LAND, RMSE_ERA5_LAND, BIAS_ERA5_LAND]
data_THERMAL=[R2_THERMAL, MAE_THERMAL, RMSE_THERMAL, BIAS_THERMAL]
data_MODIS=[R2_MODIS, MAE_MODIS, RMSE_MODIS, BIAS_MODIS]

# Iterate over metrics and create plots
for metric, data_era5, data_era5_land, data_thermal, data_modis in zip(metrics, data_ERA5, data_ERA5_LAND,data_THERMAL, data_MODIS):
    plt.figure(figsize=(12, 5))
    plt.scatter(x, data_era5, label='ERA5')
    for i, location in enumerate(L):
        plt.plot([i, i], [0, data_era5[i]], color='black', linestyle='-')
    plt.scatter(x, data_era5_land, label='ERA5-LAND')
    for i, location in enumerate(L):
        plt.plot([i, i], [0, data_era5_land[i]], color='black', linestyle='-')
    plt.scatter(x,data_thermal, label='VNP21A1D')
    for i,location in enumerate(L):
        plt.plot([i,i],[0,data_thermal[i]], color='black', linestyle='-')
    plt.scatter(x,data_modis, label='MODIS')
    for i,location in enumerate(L):
        plt.plot([i,i],[0,data_modis[i]], color='black', linestyle='-')
    plt.xlabel('Climate Stations')
    plt.ylabel(metric)
    if metric == 'R2':
        plt.ylim(min(min(data_era5_land), min(data_era5),min(data_thermal),min(data_modis)) - 0.01, max(max(data_era5_land), max(data_era5),max(data_thermal),max(data_modis)) + 0.01)
    else:
        plt.ylim(min(min(data_era5_land), min(data_era5),min(data_thermal),min(data_modis)) - 0.1, max(max(data_era5_land), max(data_era5),max(data_thermal),max(data_modis)) + 0.1)
    plt.axhline(y=0, color='black', linestyle='-')
    plt.title(metric + ' ' + Parameter)
    plt.xticks(x, L, rotation=90)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
#Manitoba territory
Parameter= 'Daily Skin Temperature'

locations = [
    {'name': "alonsa",'latitude':50.70 , 'longitude': -99.07, 'climateID': None}, 
    {'name': "altona",'latitude':49.10 , 'longitude': -97.53, 'climateID': None},
    {'name': "arborg",'latitude':50.90 , 'longitude': -97.27, 'climateID': None},
    {'name': "argue",'latitude':49.44 , 'longitude': -100.31, 'climateID': None},
    {'name': "baldur",'latitude':49.36, 'longitude': -99.25, 'climateID': None},
    {'name': "boissevain",'latitude': 49.24, 'longitude': -100.06, 'climateID': None},
    {'name': "carman",'latitude':49.50 , 'longitude': -98.03, 'climateID': None},
    {'name': "cartwright",'latitude':49.09 , 'longitude': -99.32, 'climateID': None},
    {'name': "deloraine",'latitude': 49.15, 'longitude': -100.49, 'climateID': None},
    {'name': "elmcreek",'latitude': 49.68, 'longitude': -97.98, 'climateID': None},
    {'name': "ethelbert",'latitude':  51.57, 'longitude': -100.49, 'climateID': None},
    {'name': "findlay",'latitude': 49.55, 'longitude': -100.78, 'climateID': None},
    {'name': "fisherton",'latitude': 51.18, 'longitude': -97.77, 'climateID': None},
    {'name': "Forkriver",'latitude':51.54 , 'longitude': -100.01, 'climateID': None},
    {'name': "forrest",'latitude': 50.05, 'longitude': -99.93, 'climateID': None},
    {'name': "gladstone",'latitude': 50.14, 'longitude': -98.95, 'climateID': None},
    {'name': "glenboro",'latitude': 49.55, 'longitude': -99.33, 'climateID': None},
    {'name': "inwood",'latitude':50.51 , 'longitude': -97.62, 'climateID': None}, 
    {'name': "manitou",'latitude':49.25 , 'longitude':-98.55, 'climateID': None},
    {'name': "menisino",'latitude':49.10 , 'longitude': -96.14, 'climateID': None},
    {'name': "minitonas",'latitude': 52.14, 'longitude': -100.88, 'climateID': None},
    {'name': "minto",'latitude': 49.43, 'longitude':-99.92, 'climateID': None},
    {'name': "Mountainside",'latitude': 49.13, 'longitude':-100.30 , 'climateID': None},
    {'name': "Snowflake",'latitude':49.02 , 'longitude':-98.67 , 'climateID': None},
    {'name': "somerset",'latitude':49.41 , 'longitude': -98.70, 'climateID': None},
    {'name': "souris",'latitude': 49.63, 'longitude': -100.20, 'climateID': None},
    {'name': "starbuck",'latitude':49.77 , 'longitude':-97.66 , 'climateID': None},
    {'name': "steinbach",'latitude': 49.55, 'longitude':-96.68 , 'climateID': None},
    {'name': "swanvalley",'latitude':52.06 , 'longitude': -101.49, 'climateID': None},
    {'name': "Taylorspoint",'latitude':50.86 , 'longitude':-98.45 , 'climateID': None},
    {'name': "teulon",'latitude':50.38 , 'longitude': -97.24, 'climateID': None},
    {'name': "treherne",'latitude':49.63 , 'longitude': -98.68, 'climateID': None},
    {'name': "waskada",'latitude':49.09 , 'longitude':-100.93 , 'climateID': None},
    {'name': "wawanesa",'latitude':49.64 , 'longitude': -99.66, 'climateID': None},
    {'name': "windygates",'latitude':49.01 , 'longitude':-98.38 , 'climateID': None},
]

L=[]
R2_ERA5=[]
MAE_ERA5=[]
RMSE_ERA5=[]
BIAS_ERA5=[]
R2_ERA5_LAND=[]
MAE_ERA5_LAND=[]
RMSE_ERA5_LAND=[]
BIAS_ERA5_LAND=[]
R2_THERMAL=[]
MAE_THERMAL=[]
RMSE_THERMAL=[]
BIAS_THERMAL=[]
R2_MODIS=[]
MAE_MODIS=[]
RMSE_MODIS=[]
BIAS_MODIS=[]

# Loop over each location and plot temperature data
for location in locations:
    try:
    #ERA5-LAND - 11.1 km
        df_era5_land = extract_values(location['latitude'], location['longitude'], '1950-01-02', '2024-01-31','ECMWF/ERA5_LAND/DAILY_AGGR', 'skin_temperature')
        df_era5_land_dates=pd.to_datetime(df_era5_land['date'])
        df_era5_land_value=df_era5_land['value']-273.15
        df_era5_land_dates, df_era5_land_value = zip(*sorted(zip(df_era5_land_dates, df_era5_land_value)))
        #ERA5 - 27.8 km
        years = list(range(2000, 2025))
        skin_temperature_by_year = []
        for year in years:
            ds_year = xr.open_dataset(f'era5_skt{year}.grib', engine='cfgrib')
            ds_year_by_day = ds_year['skt'].resample(time='D').mean()
            skin_temperature_by_year.append(ds_year_by_day)
            ds_year.close()
        skin_temperature_combined = xr.concat(skin_temperature_by_year, dim='time')
        skin_temperature = skin_temperature_combined.sel(latitude=location['latitude'], longitude=location['longitude'], method='nearest') - 273.15
        df_era5_dates= pd.to_datetime(skin_temperature.time)
        df_era5_value=skin_temperature.values
        df_era5_dates, df_era5_value = zip(*sorted(zip(df_era5_dates, df_era5_value)))
        #INFO THERMAL - 1 km
        df_thermal = extract_values(location['latitude'], location['longitude'], '2012-01-19', '2024-03-01','NOAA/VIIRS/001/VNP21A1D','LST_1KM')
        df_thermal_dates=pd.to_datetime(df_thermal['date'])
        df_thermal_value=df_thermal['value']-273.15
        df_thermal_dates, df_thermal_value = zip(*sorted(zip(df_thermal_dates, df_thermal_value)))
        #INFO MODIS - 1 km
        df_modis=extract_values(location['latitude'], location['longitude'], '2000-02-24', '2024-03-24','MODIS/061/MOD11A1','LST_Day_1km')
        df_modis_dates=pd.to_datetime(df_modis['date'])
        df_modis_value=(df_modis['value']*0.02)-273.15
        df_modis_dates, df_modis_value = zip(*sorted(zip(df_modis_dates, df_modis_value)))
        #CLIMATE STATIONS
        file_path = f"{location['name']}_data.txt"    
        data = pd.read_csv(file_path, sep=",")
        data["date"] = pd.to_datetime(data["date"])
        station_dates = data["date"]
        station_value = data["avg_soil_t05"]
        station_dates, station_value = zip(*sorted(zip(station_dates, station_value)))
        mask = np.isnan(station_value)
        non_nan_indices = np.logical_not(mask)
        station_value = np.array(station_value)[non_nan_indices]
        station_dates = np.array(station_dates)[non_nan_indices]
        # Crear un DataFrame con las fechas y los valores filtrados
        filtered_data = pd.DataFrame({"date": station_dates, "avg_soil_t05": station_value})
        grouped_data = filtered_data.groupby("date")["avg_soil_t05"].mean().reset_index()
        station_dates = grouped_data["date"].to_numpy()
        station_dates = pd.to_datetime(station_dates)
        station_value = grouped_data["avg_soil_t05"].to_numpy()
    
        if len(station_dates) == 0:
            continue
        else:
            #Remove dates not present in both datasets. Siempre la estacion va a tener menos data.
            common_dates_era5 = set(df_era5_dates).intersection(set(station_dates)) #se extraen las fechas que tienen iguales
            common_dates_era5_land=set(df_era5_land_dates).intersection(set(station_dates))
            common_dates_thermal=set(df_thermal_dates).intersection(set(station_dates))
            common_dates_modis=set(df_modis_dates).intersection(set(station_dates))
            
            #VALORES ERA5
            era5_dates=[]
            era5_value=[]
            for a,b in zip(df_era5_dates,df_era5_value):
                if a in common_dates_era5:
                    era5_dates.append(a)
                    era5_value.append(b)
            st_dates_era5=[]
            st_value_era5=[]
            for a,b in zip(station_dates,station_value):
                if a in common_dates_era5:
                    st_dates_era5.append(a)
                    st_value_era5.append(b)
        
            #VALORES ERA5-LAND
            era5_land_dates=[]
            era5_land_value=[]
            for a,b in zip(df_era5_land_dates,df_era5_land_value):
                if a in common_dates_era5_land:
                    era5_land_dates.append(a)
                    era5_land_value.append(b)
            st_dates_era5_land=[]
            st_value_era5_land=[]
            for a,b in zip(station_dates,station_value):
                if a in common_dates_era5_land:
                    st_dates_era5_land.append(a)
                    st_value_era5_land.append(b)
               
            #VALORES THERMAL IMAGES
            thermal_dates=[]
            thermal_value=[]
            for a,b in zip(df_thermal_dates,df_thermal_value):
                if a in common_dates_thermal:
                    thermal_dates.append(a)
                    thermal_value.append(b)
            st_dates_thermal=[]
            st_value_thermal=[]
            for a,b in zip(station_dates,station_value):
                if a in common_dates_thermal:
                    st_dates_thermal.append(a)
                    st_value_thermal.append(b)
    
            #VALORES MODIS
            modis_dates=[]
            modis_value=[]
            for a,b in zip(df_modis_dates,df_modis_value):
                if a in common_dates_modis:
                    modis_dates.append(a)
                    modis_value.append(b)
            st_dates_modis=[]
            st_value_modis=[]
            for a,b in zip(station_dates,station_value):
                if a in common_dates_modis:
                    st_dates_modis.append(a)
                    st_value_modis.append(b)
                    
            # Calculate metrics
            r2_era5, mae_era5, rmse_era5, bias_era5 = calculate_metrics(st_value_era5, era5_value)
            r2_era5_land, mae_era5_land, rmse_era5_land, bias_era5_land = calculate_metrics(st_value_era5_land, era5_land_value)
            r2_thermal, mae_thermal, rmse_thermal, bias_thermal=calculate_metrics(st_value_thermal, thermal_value)
            r2_modis, mae_modis, rmse_modis, bias_modis=calculate_metrics(st_value_modis, modis_value)
            #if r2_era5<0:
             #   r2_era5=np.nan
            #if r2_era5_land<0:
            #    r2_era5_land=np.nan
            print(f"Metrics ERA5 for {location['name']} - R^2: {r2_era5:.2f}, MAE: {mae_era5:.2f}, RMSE: {rmse_era5:.2f}, Bias: {bias_era5:.2f}")
            print(f"Metrics ERA5-LAND for {location['name']} - R^2: {r2_era5_land:.2f}, MAE: {mae_era5_land:.2f}, RMSE: {rmse_era5_land:.2f}, Bias: {bias_era5_land:.2f}")
            print(f"Metrics VNP21A1D for {location['name']} - R^2: {r2_thermal:.2f}, MAE: {mae_thermal:.2f}, RMSE: {rmse_thermal:.2f}, Bias: {bias_thermal:.2f}")
            print(f"Metrics MODIS for {location['name']} - R^2: {r2_modis:.2f}, MAE: {mae_modis:.2f}, RMSE: {rmse_modis:.2f}, Bias: {bias_modis:.2f}")
            L.append(location['name'])
            R2_ERA5.append(r2_era5)
            MAE_ERA5.append(mae_era5)
            RMSE_ERA5.append(rmse_era5)
            BIAS_ERA5.append(bias_era5)
            R2_ERA5_LAND.append(r2_era5_land)
            MAE_ERA5_LAND.append(mae_era5_land)
            RMSE_ERA5_LAND.append(rmse_era5_land)
            BIAS_ERA5_LAND.append(bias_era5_land)
            R2_THERMAL.append(r2_thermal)
            MAE_THERMAL.append(mae_thermal)
            RMSE_THERMAL.append(rmse_thermal)
            BIAS_THERMAL.append(bias_thermal)
            R2_MODIS.append(r2_modis)
            MAE_MODIS.append(mae_modis)
            RMSE_MODIS.append(rmse_modis)
            BIAS_MODIS.append(bias_modis)
        
    except ValueError as e:
        print(f"Error procesando el archivo para {location['name']}: {e}")
        # Aquí puedes optar por realizar alguna acción adicional, como registrar el error
        continue  # Continuar con la siguiente iteración del bucle

    # Si necesitas agregar más excepciones que puedan ocurrir, puedes agregarlas aquí
    except FileNotFoundError as e:
        print(f"Archivo no encontrado para {location['name']}: {e}")
        continue
    except Exception as e:
        print(f"Se produjo un error inesperado para {location['name']}: {e}")
        continue


# Set the positions of the points on the x-axis
x = np.arange(len(L))

# Define the metrics and corresponding data lists
metrics = ['R2', 'MAE', 'RMSE', 'BIAS']
data_ERA5 = [R2_ERA5, MAE_ERA5, RMSE_ERA5, BIAS_ERA5]
data_ERA5_LAND = [R2_ERA5_LAND, MAE_ERA5_LAND, RMSE_ERA5_LAND, BIAS_ERA5_LAND]
data_THERMAL=[R2_THERMAL, MAE_THERMAL, RMSE_THERMAL, BIAS_THERMAL]
data_MODIS=[R2_MODIS, MAE_MODIS, RMSE_MODIS, BIAS_MODIS]

# Iterate over metrics and create plots
for metric, data_era5, data_era5_land, data_thermal, data_modis in zip(metrics, data_ERA5, data_ERA5_LAND,data_THERMAL, data_MODIS):
    plt.figure(figsize=(12, 5))
    plt.scatter(x, data_era5, label='ERA5')
    for i, location in enumerate(L):
        plt.plot([i, i], [0, data_era5[i]], color='black', linestyle='-')
    plt.scatter(x, data_era5_land, label='ERA5-LAND')
    for i, location in enumerate(L):
        plt.plot([i, i], [0, data_era5_land[i]], color='black', linestyle='-')
    plt.scatter(x,data_thermal, label='VNP21A1D')
    for i,location in enumerate(L):
        plt.plot([i,i],[0,data_thermal[i]], color='black', linestyle='-')
    plt.scatter(x,data_modis, label='MODIS')
    for i,location in enumerate(L):
        plt.plot([i,i],[0,data_modis[i]], color='black', linestyle='-')
    plt.xlabel('Climate Stations')
    plt.ylabel(metric)
    if metric == 'R2':
        plt.ylim(min(min(data_era5_land), min(data_era5),min(data_thermal),min(data_modis)) - 0.01, max(max(data_era5_land), max(data_era5),max(data_thermal),max(data_modis)) + 0.01)
    else:
        plt.ylim(min(min(data_era5_land), min(data_era5),min(data_thermal),min(data_modis)) - 0.1, max(max(data_era5_land), max(data_era5),max(data_thermal),max(data_modis)) + 0.1)
    plt.axhline(y=0, color='black', linestyle='-')
    plt.title(metric + ' ' + Parameter)
    plt.xticks(x, L, rotation=90)
    plt.legend()
    plt.tight_layout()
    plt.show()